# Reusable code for ride-sharing optimization

Linear optimization formulation to solve a simplified version of the pooling problem faced by ride-hailing companies such as Uber and Lyft. These companies provide discounts to riders who are willing to let their trips be "pooled" with another trip, so that a driver would pick up both riders before dropping off each of them at their respective destinations. Through pooling, the travel time of each rider would increase, but driver capacity is more efficiently utilized. This practice is only profitable if the company finds good matches between trips so as to maximize the total benefit of pooling. 

For simplicity, assume that each trip can be pooled with at most one other trip. The ride-hailing company has estimated the benefit of pooling for each pair of trips, which accounts for the potential cost savings from pooling and the potential inconveniences incurred for the pooled customers. As an illustration, suppose there are six trips, labelled A through F. The following table shows what the benefit values may look like.

|Benefit of Pooling | A | B | C | D | E | F |
|--|--|--|--|--|--|--|
| A | 0 | 6 | 4 | 3 | 1 | 1 | 
| B | 6 | 0 | 5 | 5 | 2 | 3 |
| C | 4 | 5 | 0 | 1 | 4 | 3 |
| D | 3 | 5 | 1 | 0 | 2 | 1 |
| E | 1 | 2 | 4 | 2 | 0 | 4 |
| F | 1 | 3 | 3 | 1 | 4 | 0 |

The ride-hailing company would like to pool trips so as to maximize the total benefit of pooled trips subject to the following constraints:

- Each trip can be pooled with at most one other trip. (It is also possible for a trip to be not pooled)
- For a pair of trips, if the benefit of pooling is strictly less than a threshold $t$, then the trips cannot be pooled. For example, if $t=3$, then the above table implies that A-D is a valid pooling, but B-E is not. 
- The total number of pooled pairs is at most $k$. In the above example, if $t=0$ and $k=3$, then the optimal solution is to pool A-C, B-D, and E-F, which yields a total benefit of $4+5+4=13$. However, if $t=0$ and $k=1$, then the optimal solution is to only pool A-B, which yields a benefit of $6$.

Assume that $t$ is always a non-negative number, and $k$ is always a positive integer. 

### Abstract formulation

**Data:**

- $S$: set of trips.
- $v_{ij}$: the benefit of pooling trip $i \in S$ with trip $j \in S$. 
- $t$: the threshold on the benefit of pooling below which pooling for the given pair is not allowed.
- $k$: the maximum number of pooled pairs allowed.

**Decision Variables:**

- $x_{ij}$: whether to pool trip $i \in S$ with trip $j \in S$. (Binary) 

**Objective:**

$$\text{Maximize: } 0.5\sum_{i \in S, j \in S} v_{ij}x_{ij} $$

**Constraints:**

$$\begin{aligned}
\sum_{j \in S}x_{ij} & \le 1 && \text{for each trip $i \in S$.} \\
x_{ii} & = 0 && \text{for each trip $i \in S$.} \\
x_{ij} & = x_{ji} && \text{for each pair $i\in S, j \in S$.} \\
x_{ij} & = 0  && \text{for each pair $i\in S, j \in S$ such that $v_{ij}<t$.} \\
0.5\sum_{i \in S, j \in S}x_{ij} & \le k
\end{aligned}$$

### Gurobi Code

In [1]:
import pandas as pd
from gurobipy import Model, GRB

In [2]:
def rideshare(inputFile,outputfile):
#     inputFile='Rideshare.xlsx'
#     outputfile='Solution.xlsx'
    v=pd.read_excel(inputFile,sheet_name='Benefit of Pooling',index_col=0)
    t=pd.read_excel(inputFile,sheet_name='Threshold').loc[0,'Threshold']
    k=pd.read_excel(inputFile,sheet_name='Pool Limit').loc[0,'Maximum number of pooled pairs']
    mod=Model()
    I=v.index
    J=v.columns
    x=mod.addVars(I,J,name='x',vtype=GRB.BINARY)
    mod.update()
    mod.setObjective(0.5* sum(v.loc[i,j]* x[i,j] for i in I for j in J),sense=GRB.MAXIMIZE)
    for i in I:
        mod.addConstr(sum(x[i,j] for j in J)<=1)
        mod.addConstr(x[i,i]==0)
        for j in J:
            mod.addConstr(x[i,j]==x[j,i])
            if v.loc[i,j]<t:
                mod.addConstr(x[i,j]==0)
    mod.addConstr(0.5*sum(x[i,j] for i in I for j in J)<=k)
    mod.setParam('OutputFlag', False)
    mod.optimize()
    optimized_table=pd.DataFrame(index=I,columns=J)
    pool=[]
    for i in I:
        for j in J:
            optimized_table.loc[i,j]=round(x[i,j].x)
            if  x[i,j].x:
                pool.append([i,j])
    final = set()
    upd=[x for x in pool if frozenset(x) not in final and not final.add(frozenset(x))]
    objective=pd.DataFrame([mod.objval],columns=['Optimum benefit'])                
    selection=pd.DataFrame(upd,columns=['Pool','With'])
    selection
    writer=pd.ExcelWriter(outputfile)
    selection.to_excel(writer,sheet_name='optimal_pooling',index=False)
    objective.to_excel(writer,sheet_name='objective',index=False)
    writer.save()

In [3]:
#Sample run
rideshare('Rideshare.xlsx','Solution.xlsx')

Set parameter Username
Academic license - for non-commercial use only - expires 2022-01-29
